In [1]:
import time
import numpy as np
from gep_config_parser import *
from data_wrangling import dataframe_to_dict
import pyomo.environ as pyo

from gep_main import run_model, prep_data, run_model_no_bounds
from gep_problem import GEPProblem
from gep_problem_refactored import GEPProblemSet

import torch

DTYPE = torch.float64

torch.set_default_dtype(DTYPE)

# DEVICE = (
#     torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
# )

DEVICE = "cpu"


CONFIG_FILE_NAME        = "config.toml"
VISUALIZATION_FILE_NAME = "visualization.toml"

HIGHS  = "HiGHS"
GUROBI = "Gurobi"

SAMPLE_DURATION = 24

## Step 0: Activate environment - ensure consistency accross computers
# print("Reading the data")
# print("Activating the environment")

## Step 1: parse the input data
print("Parsing the config file")

data = parse_config(CONFIG_FILE_NAME)
experiment = data["experiment"]
outputs_config = data["outputs_config"]

print("Initializing the solver")
optimizer_name = data["optimizer_config"]["solver"]

# Determine the optimizer
if optimizer_name == HIGHS:
    raise NotImplementedError(f"{optimizer_name}: Not implemented")
elif optimizer_name == GUROBI:
    
    print(f"Using {GUROBI}")
    optimizer = "gurobi_direct"
else:
    raise ValueError(f"{optimizer_name}: Not implemented")

Parsing the config file
Initializing the solver
Using Gurobi
Parsing the config file
Initializing the solver
Using Gurobi


In [2]:
for i, experiment_instance in enumerate(experiment["experiments"]):
    # Setup output dataframe
    df_res = pd.DataFrame(columns=["setup_time", "presolve_time", "barrier_time", "crossover_time", "restore_time", "objective_value"])

    input_T, input_N, input_G, input_L, input_pDemand, input_pGenAva, input_pVOLL, input_pWeight, input_pRamping, input_pInvCost, input_pVarCost, input_pUnitCap, input_pExpCap, input_pImpCap = prep_data(experiment_instance)

    T_ranges = [range(i, i + SAMPLE_DURATION, 1) for i in range(1, len(input_T), SAMPLE_DURATION)]
    objective_values = []
    times = []
    models = []
    solvers = []
    for t in T_ranges:
        # Run one experiment for j repeats
        model, solver, time_taken = run_model(experiment_instance, t, input_N, input_G, input_L, input_pDemand, input_pGenAva, input_pVOLL, input_pWeight, input_pRamping, input_pInvCost, input_pVarCost, input_pUnitCap, input_pExpCap, input_pImpCap)
        models.append(model)
        solvers.append(solver)

Wrangling the input data
Populating the model
Adding model variables
Formulating the objective
Adding model constraints
Solving the optimization problem
Set parameter OutputFlag to value 1
Set parameter LogFile to value "outputs/Gurobi/output.txt"
Set parameter Crossover to value 0
Set parameter FeasibilityTol to value 1e-09
Set parameter QCPDual to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 22.3.0 22D49)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
FeasibilityTol  1e-09
Method  2
Crossover  0
QCPDual  1

Optimize a model with 284 rows, 221 columns and 947 nonzeros
Model fingerprint: 0x0d0f9a5a
Coefficient statistics:
  Matrix range     [7e-02, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+03, 4e+04]
  RHS range        [4e+03, 4e+04]
Presolve removed 244 rows and 192 columns
Presolve time: 0.00s
Presolved: 40 rows, 29 columns, 105 nonzeros
Ordering ti

In [3]:
def get_variable_values_as_list(var):
    # Check if the variable is indexed
    if var.is_indexed():
        return [var[idx].value for idx in var]
    else:
        # For scalar variables
        return [var.value]

# Function to get constraint values as a list
def get_constraint_values_as_list(constraint):
    constraint_values = []
    for idx in constraint:
        # Evaluate the constraint body
        constraint_expr_value = pyo.value(constraint[idx].body)
        constraint_values.append(constraint_expr_value)
    return constraint_values

def get_variable_bounds(var):
    lower_bounds = []
    upper_bounds = []
    
    # Check if the variable is indexed
    if var.is_indexed():
        # Iterate through the indices of the variable
        for idx in var:
            lower_bounds.append(var[idx].lb)  # Get lower bound for each index
            upper_bounds.append(var[idx].ub)  # Get upper bound for each index
    else:
        # For scalar variables
        lower_bounds.append(var.lb)  # Get lower bound of scalar variable
        upper_bounds.append(var.ub)  # Get upper bound of scalar variable

    return lower_bounds, upper_bounds

In [4]:
#T, N, G, L, pDemand, pGenAva, pVOLL, pWeight, pRamping, pInvCost, pVarCost, pUnitCap, pExpCap, pImpCap
data = GEPProblemSet(input_T, input_N, input_G, input_L, input_pDemand, input_pGenAva, input_pVOLL, input_pWeight, input_pRamping, input_pInvCost, input_pVarCost, input_pUnitCap, input_pExpCap, input_pImpCap, sample_duration=SAMPLE_DURATION)

eq_cm = data.eq_cm
ineq_cm = data.ineq_cm
eq_rhs = data.eq_rhs
ineq_rhs = data.ineq_rhs
obj_coeff = data.obj_coeff
# X contains [pGenAva, pDemand]
all_X = data.X


# Prepare batched inputs
X = all_X  # Adjust for batched input
Y_list = [
    torch.tensor(
        [[*get_variable_values_as_list(model.vGenProd),
          *get_variable_values_as_list(model.vLineFlow),
          *get_variable_values_as_list(model.vLossLoad),
          *get_variable_values_as_list(model.vGenInv)]],
        device=DEVICE
    ) for model in models
]
Y = torch.cat(Y_list, dim=0)  # Batch all Y tensors

print(f"Y: {Y.shape}")
# print(Y)
print(f"ineq_cm: {ineq_cm.shape}")
print(f"ineq_rhs: {ineq_rhs.shape}")



Populating ineq constraints


Populating eq constraints
Creating objective coefficients
Creating input for NN: X
Y: torch.Size([365, 219])
ineq_cm: torch.Size([365, 573, 219])
ineq_rhs: torch.Size([365, 573])


In [22]:
def ineq_resid(Y, ineq_cm, ineq_rhs):
    return torch.bmm(ineq_cm, Y.unsqueeze(-1)).squeeze(-1) - ineq_rhs

def eq_resid(Y, eq_cm, eq_rhs):
    return torch.bmm(eq_cm, Y.unsqueeze(-1)).squeeze(-1) - eq_rhs

def obj_fn(Y, obj_coeff):
    # obj_coeff does not need batching, objective is the same over different samples.
    return obj_coeff @ Y.T

In [19]:
print("### Testing consistency of g_x(y) ###")
# Compute g in a batched way
g_batch = ineq_resid(Y, ineq_cm, ineq_rhs).squeeze(1).tolist()

# Process constraints in batch
eMaxProd_idx = len(input_G) * SAMPLE_DURATION
eMaxProd_PDL = [g[:eMaxProd_idx] for g in g_batch]
eMaxProd_LP_list = [get_constraint_values_as_list(model.eMaxProd) for model in models]

# Check eMaxProd consistency
for g_pdl, g_lp in zip(eMaxProd_PDL, eMaxProd_LP_list):
    assert np.allclose(g_pdl, g_lp, atol=1e-9)

# Process other constraints in batch
for i, (g, model) in enumerate(zip(g_batch, models)):
    print(i)
    eLineFlow_idx_lb = eMaxProd_idx + len(input_L) * SAMPLE_DURATION
    eLineFlow_idx_ub = eLineFlow_idx_lb + len(input_L) * SAMPLE_DURATION

    eLineFlow_PDL_lb = np.array(g[eMaxProd_idx:eLineFlow_idx_lb], dtype=np.float64)
    eLineFlow_PDL_ub = np.array(g[eLineFlow_idx_lb:eLineFlow_idx_ub], dtype=np.float64)

    eLineFlow_LP_lb, eLineFlow_LP_ub = (
        np.array(get_variable_bounds(model.vLineFlow)[0], dtype=np.float64),
        np.array(get_variable_bounds(model.vLineFlow)[1], dtype=np.float64),
    )
    vLineFlow = np.array(get_variable_values_as_list(model.vLineFlow), dtype=np.float64)

    # assert np.allclose(eLineFlow_PDL_lb + vLineFlow, eLineFlow_LP_lb, atol=1e-9)
    assert np.allclose(
        eLineFlow_PDL_lb + vLineFlow, eLineFlow_LP_lb, atol=1e-9
    ), f"Assertion failed!\nvLineFlow:\n{vLineFlow.tolist()}\neLineFlow_PDL_lb:\n{eLineFlow_PDL_lb.tolist()}\neLineFlow_PDL_lb + vLineFlow:\n{(eLineFlow_PDL_lb + vLineFlow).tolist()}\neLineFlow_LP_lb:\n{eLineFlow_LP_lb.tolist()}\nDifference:\n{(eLineFlow_PDL_lb + vLineFlow - eLineFlow_LP_lb).tolist()}"
    assert np.allclose(-1 * (eLineFlow_PDL_ub - vLineFlow), eLineFlow_LP_ub, atol=1e-9)

    # Continue with other constraint tests similarly...
    eRampingDown_idx = eLineFlow_idx_ub + len(input_G) * (SAMPLE_DURATION - 1)
    eRampingDown_PDL = np.array(g[eLineFlow_idx_ub:eRampingDown_idx])
    eRampingDown_LP = np.array(get_constraint_values_as_list(model.eRampingDown))

    eRampingUp_idx = eRampingDown_idx + len(input_G) * (SAMPLE_DURATION - 1)
    eRampingUp_PDL = np.array(g[eRampingDown_idx:eRampingUp_idx])
    eRampingUp_LP = np.array(get_constraint_values_as_list(model.eRampingUp))

    assert np.allclose(
        eRampingDown_PDL, eRampingDown_LP, atol=1e-9
    ), f"Assertion failed!\neRampingDown_PDL:\n{eRampingDown_PDL.tolist()}\nrRampingDown_LP:\n{eRampingDown_LP.tolist()}\nDifference:\n{(eRampingDown_PDL - eRampingDown_LP).tolist()}"

    assert np.allclose(np.array(eRampingUp_PDL), np.array(eRampingUp_LP), atol=1e-9)

    eGenProdPositive_idx = eRampingUp_idx + len(input_G) * SAMPLE_DURATION
    eGenProdPositive_PDL = g[eRampingUp_idx:eGenProdPositive_idx]
    vGenProd = get_variable_values_as_list(model.vGenProd)

    assert np.allclose(np.array(eGenProdPositive_PDL), -1 * np.array(vGenProd), atol=1e-9)

    eMissedDemandPositive_idx = eGenProdPositive_idx + len(input_N) * SAMPLE_DURATION
    eMissedDemandPositive_PDL = np.array(g[eGenProdPositive_idx:eMissedDemandPositive_idx])
    vLossLoad = np.array(get_variable_values_as_list(model.vLossLoad))

    assert np.allclose(
        eMissedDemandPositive_PDL, -vLossLoad, atol=1e-9
    ),f"Assertion failed!\neMissedDemandPositive_PDL:\n{eMissedDemandPositive_PDL.tolist()}\n-vLossLoad:\n{(-vLossLoad).tolist()}\nDifference:\n{(eMissedDemandPositive_PDL + vLossLoad).tolist()}" 

    # 3.1j
    eMissedDemandLeqDemand_idx = eMissedDemandPositive_idx + len(input_N) * SAMPLE_DURATION
    eMissedDemandLeqDemand_PDL = np.array(g[eMissedDemandPositive_idx:eMissedDemandLeqDemand_idx])
    vLossLoad = np.array(get_variable_values_as_list(model.vLossLoad))
    # First N*T of each sample is the demand.
    demand = data.X[i, :len(input_N)*SAMPLE_DURATION].numpy()

    assert np.allclose(
        eMissedDemandLeqDemand_PDL,  vLossLoad - demand, atol=1e-9
    ),f"Assertion failed!\eMissedDemandLeqDemand:\n{(eMissedDemandLeqDemand_PDL).tolist()}\n-vLossLoad:\n{(vLossLoad - demand).tolist()}\nDifference:\n{(eMissedDemandLeqDemand_PDL - (vLossLoad - demand)).tolist()}" 

    eNonNegativeGenInv_PDL = np.array(g[eMissedDemandLeqDemand_idx:])
    vGenInv = np.array(get_variable_values_as_list(model.vGenInv), dtype=np.float64)

    assert np.allclose(
        eNonNegativeGenInv_PDL, -vGenInv, atol=1e-9
    ),f"Assertion failed!\eNonNegativeGenInv_PDL:\n{(eNonNegativeGenInv_PDL).tolist()}\n-vGenInv:\n{(-vGenInv).tolist()}\nDifference:\n{(eNonNegativeGenInv_PDL + vGenInv).tolist()}" 



### Testing consistency of g_x(y) ###
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267


In [20]:
print("### Testing consistency of h_x(y) ###")
# Compute h in a batched way
h_batch = eq_resid(Y, eq_cm, eq_rhs).squeeze(1).tolist()

# Prepare eNodeBal and pDemand for all models
eNodeBal_LP_list = [get_constraint_values_as_list(model.eNodeBal) for model in models]
pDemand_list = data.X[:, :len(input_N)*SAMPLE_DURATION].tolist()


# Check eNodeBal consistency in batch
for h, eNodeBal_LP, pDemand in zip(h_batch, eNodeBal_LP_list, pDemand_list):
    assert np.allclose(
        np.array(h), 
        np.array(eNodeBal_LP) - np.array(pDemand), 
        atol=1e-8
    ), (
        f"Assertion failed: eNodeBal_PDL != eNodeBal_LP - pDemand\n"
        f"  h (PDL): {h}\n"
        f"  eNodeBal_LP: {eNodeBal_LP}\n"
        f"  pDemand: {pDemand}"
    )

### Testing consistency of h_x(y) ###


In [23]:
print("### Testing consistency of f_x(y) ###")

# Compute objective function values for all models
obj_LP_list = [model.obj() for model in models]
obj_PDL_list = obj_fn(Y, obj_coeff).tolist()

# Check objective function consistency in batch
for obj_LP, obj_PDL in zip(obj_LP_list, obj_PDL_list):
    assert abs((obj_LP - obj_PDL) / obj_LP) < 1e-9, (
        f"Assertion failed: Objective mismatch\n"
        f"  obj_LP: {obj_LP}\n"
        f"  obj_PDL: {obj_PDL}\n"
        f"  Relative difference: {abs((obj_LP - obj_PDL) / obj_LP)}"
    )

### Testing consistency of f_x(y) ###
